<a href="https://colab.research.google.com/github/mmeendez8/mmdet_benchmark/blob/main/mmdet_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mmdet model comparison

In this notebook we will show a simple way of running the benchmark script on different mmdetection models and compare their results.

Work is inspired by [Timm and Jeremy Howard notebook](https://www.kaggle.com/code/jhoward/which-image-models-are-best) where they compare Timm Image Model performance.

## Install torch and mmdetection

Note benchmarking is carried on GPU so be sure to enable GPU for this notebook.

In [ ]:
# Latest torch version supported by mmengine at this moment
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install mmengine==0.10.2
!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html
!pip install mmdet==3.3.0

## Download COCO val data

In [ ]:
# Create the coco directory and cd into it
!mkdir coco
%cd coco

# Create the images directory and cd into it
!mkdir images
%cd images

!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip
!rm val2017.zip

# Go back to the coco directory
%cd ../

# Download the annotation zip files
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

# Delete extra files
!find annotations -type f ! -name 'instances_val2017.json' -delete

# Clean up the zip files
!rm annotations_trainval2017.zip

# Go back to root directory
%cd ../


## Clone benchmark metadata

I have created a simple GH repo where I saved the metadata files for the benchmarking. These are mostly model config files from the mmdet repo that I have adapted to use a fixed image size and single gpu.

The structure of this repo is very simple:

- `metadata.json`: contains simple metadata about each model like family, config file path, model mAP metric, training image size, ...
- `models/`: directory where all model config files are stored
- `models/shared_config.py`: contains the dataset config that will be used for all models. COCO val data with image resized to 640x640 px-

For adding a new model to the benchmark you just need to add a new entry to the `metadata.json` file and the config file of the model to the `models` folder.

In [ ]:
!git clone https://github.com/mmeendez8/mmdet_benchmark.git

## Benchmarking Code

This code has been created from the official mmdet benchmark script that you can check in: https://github.com/open-mmlab/mmdetection/blob/main/tools/analysis_tools/benchmark.py

In [ ]:
import os
import os.path as osp

from mmengine import MMLogger
from mmengine.config import Config
from mmengine.registry import init_default_scope
from mmdet.utils.benchmark import InferenceBenchmark

def benchmark(
    config: str,
    checkpoint: str,
    work_dir: str,
    repeat_num: int = 1,
    max_iter: int = 2000,
    log_interval: int = 50,
    num_warmup: int = 5,
    fuse_conv_bn: bool = False,
    model_name: str = "mmdet",
):
    cfg = Config.fromfile(config)

    init_default_scope(cfg.get("default_scope", "mmdet"))

    log_file = osp.join(work_dir, "benchmark.log")
    os.makedirs(work_dir, exist_ok=True)

    logger = MMLogger.get_instance(model_name, log_file=log_file, log_level="INFO")

    benchmark = InferenceBenchmark(
        cfg=cfg,
        checkpoint=checkpoint,
        distributed=False,
        is_fuse_conv_bn=fuse_conv_bn,
        max_iter=max_iter,
        log_interval=log_interval,
        num_warmup=num_warmup,
        logger=logger,
    )

    benchmark.run(repeat_num)

I will define a simple function that parses the log file output from the benchmark script and extracts fps and cuda memory metrics.

In [ ]:
import re
import torch
import gc

def parse_log_results(log_file):
    with open(log_file, "r") as f:
        lines = f.readlines()

    last_lines = lines[-3:]

    fps_pattern = r"Overall fps: (\d+\.\d+)"
    cuda_memory_pattern = r"cuda memory: (\d+)"

    overall_fps = None
    cuda_memory = None

    # Search for the values in the last three lines
    for line in last_lines:
        fps_match = re.search(fps_pattern, line)
        cuda_memory_match = re.search(cuda_memory_pattern, line)

        if fps_match:
            overall_fps = float(fps_match.group(1))
        if cuda_memory_match:
            cuda_memory = int(cuda_memory_match.group(1))

    # Clean cache
    gc.collect()
    torch.cuda.empty_cache()

    return overall_fps, cuda_memory


## Run Benchmark

In [ ]:
metadata_folder = "mmdet_benchmark/"
metadata_file = "metadata.json"
models_dir = "models"

# Benchmark variables
MAX_ITER = 500
LOG_INTERVAL = 500

In [ ]:
import json

with open(osp.join(metadata_folder, metadata_file)) as f:
    metadata = json.load(f)

for family, models in metadata.items():
    for i, model in enumerate(models):
        print("="*50)
        print(f"Benchmarking model {model['name']}")
        print("="*50)

        model_dir = osp.join(metadata_folder, models_dir, model['name'])

        config = osp.join(model_dir, "config.py")
        checkpoint = model['checkpoint']

        benchmark(config=config, checkpoint=checkpoint, max_iter=MAX_ITER, work_dir=model_dir, log_interval=LOG_INTERVAL, model_name=model['name'])
        fps, cuda_memory = parse_log_results(osp.join(model_dir, "benchmark.log"))

        # Remove the log file
        # os.remove(osp.join(model_dir, "benchmark.log"))

        print(f"Overall FPS: {fps}")
        print(f"CUDA memory: {cuda_memory} MB")

        metadata[family][i]['fps'] = fps
        metadata[family][i]['cuda_memory'] = cuda_memory

## Plot results



In [ ]:
# convert data to pandas
import pandas as pd

data = []
for family, models in metadata.items():
    for model in models:
        tmp = model.copy()
        tmp['family'] = family
        tmp["img_size"] = model["img_size"][1] * model["img_size"][0]
        data.append(tmp)

df = pd.DataFrame(data)
df

Here's the results for the performance benchmark. We can see:
- x axis shows fps to process a single image
- y axis shows map metric (directly copied from mmdet metrics)
- the size of each bubble is proportional to the size of images used in training
- the color shows what family the architecture is form

In [ ]:
import plotly.express as px
import torch

w, h = 1000,800

def show_all(df, title, x):
    return px.scatter(df, width=w, height=h, size=df["img_size"], title=title,
        x=x,  y='box_AP', log_x=False, color='family', hover_name='name')

title = f"Inference speed vs. mAP [{torch.cuda.get_device_name(torch.cuda.current_device())}]"
show_all(df, title, "fps")

We can also show the cuda memory used by the models in the x axis:

In [ ]:
title = f"Cuda memory vs. mAP [{torch.cuda.get_device_name(torch.cuda.current_device())}]"
show_all(df, title, "cuda_memory")